In [1]:
import numpy as np
#from matplotlib import pyplot as plt
from tensorflow import keras
from keras.models import Sequential, load_model
#from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
#import cv2
import os
#from sklearn import utils
import json
from keras.optimizers import Adam
#from keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Cargamos los tensores y otros datos necesarios para el modelo
X_train = np.load('/content/drive/MyDrive/Colab Notebooks/X_train.npy')
#X_test = np.load('/content/drive/MyDrive/Colab Notebooks/X_test.npy')
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/y_train.npy')
#y_test = np.load('/content/drive/MyDrive/Colab Notebooks/y_test.npy')
target_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]
num_targets = len(target_names)

In [3]:
# QUINTO MODELO:
# Aumento de datos (sobre el mejor modelo)

# Modelo basado en aumento de datos sobre cualquiera de los cuatro modelos implementados anteriormente
# (se aconseja sobre el modelo que mayor score haya devuelto la función evaluate).
# Para el aumento de datos, debe emplearse lafunción ImageDataGenerator, la elección de parámetros en ImageDataGeneratores libre.

# El mejor modelo es el cuarto (Inceptionv3):
model5 = load_model('/content/drive/MyDrive/Colab Notebooks/model4.h5')

In [4]:

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

datagen.fit(X_train)

train_generator = datagen.flow(X_train, y_train, batch_size=32, subset='training')
validation_generator = datagen.flow(X_train, y_train, batch_size=32, subset='validation')


In [5]:
%%time
# Asegurar que eager execution está activado
tf.config.run_functions_eagerly(True)

# Recompilar el modelo ANTES de entrenar
model5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

hist_model5 = model5.fit(train_generator,
                    batch_size = 32,
                    epochs = 15,
                    validation_data = validation_generator,
                    callbacks  = [early_stopping])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


111/111 ━━━━━━━━━━━━━━━━━━━━ 889s 8s/step - accuracy: 0.6096 - loss: 1.0486 - val_accuracy: 0.2077 - val_loss: 2.1661
Epoch 2/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 931s 8s/step - accuracy: 0.7248 - loss: 0.7299 - val_accuracy: 0.3251 - val_loss: 1.6825
Epoch 3/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 862s 8s/step - accuracy: 0.7510 - loss: 0.6471 - val_accuracy: 0.5869 - val_loss: 1.1263
Epoch 4/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 928s 8s/step - accuracy: 0.7636 - loss: 0.6132 - val_accuracy: 0.6772 - val_loss: 0.8693
Epoch 5/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 873s 8s/step - accuracy: 0.7815 - loss: 0.5873 - val_accuracy: 0.7336 - val_loss: 0.7119
Epoch 6/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 872s 8s/step - accuracy: 0.8037 - loss: 0.5654 - val_accuracy: 0.7528 - val_loss: 0.6686
Epoch 7/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 946s 8s/step - accuracy: 0.8141 - loss: 0.5036 - val_accuracy: 0.7585 - val_loss: 0.6623
Epoch 8/15
111/111 ━━━━━━━━━━━━━━━━━━━━ 869s 8s/step - accuracy: 0.7996 - loss: 0.5291 - val_accuracy: 0.770

In [6]:
model5.save('/content/drive/MyDrive/Colab Notebooks/model5.h5')
with open('/content/drive/MyDrive/Colab Notebooks/hist_model5.json', 'w') as f:
    json.dump(hist_model5.history, f)